In [ ]:
import pandas as pd
import numpy as np

from mlpckg.util_method import *
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# select_key() argument ("1.novem/ 2.trans/ 3.bipun/ 4.paren ")
# select_key() return (target_species,key)

num = 4
target_species,key= select_key(num)

In [ ]:
#insert your path for AOO Crawling
smooth_path = "C:/Users/DAEGYEONGLEE/Desktop/ladybug_project/Data/4th_tmp/forAOO/"

In [ ]:
shap_var = pd.read_csv("./Data/3rd_manipulated_data/shap_values/"+target_species+"shap_values_top15.csv")
shap_var = list(shap_var.name.values)

In [ ]:
df_train = pd.read_csv("./Data/2nd_manipulated_data/20220808_" + str(key)+"_train_minmax.csv")
df_train = df_train.drop(df_train[(df_train[target_species + str('_18')]!=0)&(df_train['Species']!= target_species)].index).reset_index(drop=True)

In [ ]:
df_test = pd.read_csv("./Data/2nd_manipulated_data/20220808_" + str(key)+"_test_minmax.csv")
df_test = df_test[df_test.Species==target_species].reset_index(drop=True)

In [ ]:
#remain columns which you need
delivered_package = set(df_train.columns.tolist()) - set(['Source','State','Year','Latitude','Longitude','public_positional_accuracy', 
                                                    str(target_species+str('_18'))])

delivered_package = list(delivered_package)

df_train = df_train.sort_values(by='Species').reset_index(drop=True)
df_train_18 = df_train[delivered_package]


df_train_18['target'] = 0
df_train_18.loc[df_train.Species==target_species,'target']=1
df_train_18 = df_train_18.sort_values(by='target', ascending=False).reset_index(drop=True)

In [ ]:
df_train_18.rename(columns={"Species":"Species_18"},inplace=True)
df_train_18.columns = [i.replace(" ","_").replace("-","") for i in df_train_18.columns]
    

In [ ]:
from tqdm import tqdm
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from collections import Counter

def AOO(df_train,df_test,targetnum,varlist,target_species,Year_start,Year_end,
bagstart=0,bagnum=50,repstart=0,repnum=50,learningrate=0.7,judge_indicator=0.5,to_csv=False):
    hot = list(range (bagstart, bagnum))
    pot = list(range(repstart, repnum))
    Year_end += 1
    for i in range(Year_start,Year_end):
            globals()['potcast_{}'.format(i)] = []
            
            
    for k in tqdm(hot):
        BOX = df_train.reset_index(drop = True).copy()
        absence_pool = BOX[targetnum:].sample(n=targetnum, random_state = k)
        Ml_pool = pd.concat([BOX[:targetnum], absence_pool])
        answer = Ml_pool.target
        Ml_pool = Ml_pool[varlist]



        for i in pot:
            X_train, X_test, y_train, y_test = train_test_split(Ml_pool, answer, test_size=0.000000000001, random_state= i)  
            xgb_wrapper = XGBClassifier(n_estimators=1000, learning_rate=learningrate, max_depth=7, objective = "binary:logistic")
            evals = [(X_test, y_test)]
            xgb_wrapper.fit(X_train, y_train, early_stopping_rounds=10, 
                                eval_metric="error", eval_set=evals, verbose=0)

            
            for year in range(Year_start,Year_end):
                nameind={}
                
                # Alert!
                # test csv를 만들때 실패했던 종들은 따로 코딩해줘야함.
                # When making test csv file, the failed species should be coded separately.
                for name in varlist: 
                    if name =='Coelophora_maculata_18':
                        nameind['phoramaculata_'+str(year)+"_18km"] = name

                    elif name =='Epilachna_borealis_18':
                        nameind['achnaborealis_'+str(year)+"_18km"] = name

                    elif name =='Coleomegilla_maculata_18':
                        nameind['gillamaculata_'+str(year)+"_18km"] = name

                    elif name =='Psyllobora_borealis_18':
                        nameind['boraborealis_'+str(year)+"_18km"] = name

                    elif name =='Olla_vnigrum_18':
                        nameind['v-nigrum_'+str(year)+"_18km"] = name
                    else:
                        nameind[(name.split("_")[1]+"_"+str(year)+"_18km")] = name

                df09 = df_test[list(nameind.keys())].reset_index(drop=True)


                df09.columns = list(nameind.values())
                preds_proba = xgb_wrapper.predict_proba(df09)
                presence_index = [p for p in range(len(preds_proba)) if preds_proba[p][1]>0.5]

                
                globals()["potcast_{}".format(year)].extend(presence_index)

    judgebythis = len(hot)*len(pot)*judge_indicator
    

    for year in range(Year_start,Year_end):
        globals()['count{}'.format(year)] = Counter(globals()['potcast_{}'.format(year)])          
        globals()['idx_{}'.format(year)] = []
        for i in globals()['count{}'.format(year)].most_common(targetnum):
            if i[1]>=judgebythis:
                globals()['idx_{}'.format(year)].append(i[0])

    df_test['occurrenceRemarks'] = 'Note'
    df_test.rename(columns={"Latitude":"latitude",
                   "Longitude":"longitude"},inplace=True)

   

    for i in range(Year_start,Year_end):
        globals()['df_{}'.format(i)]=df_test.iloc[globals()['idx_{}'.format(i)]][['latitude','longitude','Species','occurrenceRemarks','Year']].rename({'Species':'scientificname'},axis=1).reset_index(drop=True)

    df_modi = df_test[['latitude','longitude','Species','occurrenceRemarks','Year']].rename({'Species':'scientificname'},axis=1)

    # 머신은 없다고 판단했으나, 실제로는 있는 경우와 합침 (aoo니까)
    # Even in areas where machine learning predictions indicate that there is no target species, there may be target species in the actual identification record. 
    # In order to estimate AOO, we need to fix this case.

    for i in range(Year_start,Year_end):
        globals()['df_fin{}'.format(i)] = pd.concat([globals()['df_{}'.format(i)],df_modi[df_modi.Year==i]]).drop_duplicates(keep='first').reset_index(drop=True)
        print(i,"",len(globals()['df_fin{}'.format(i)]))
    if to_csv == True:
        for i in range(Year_start,Year_end):
            globals()['df_fin{}'.format(i)].to_csv("./Data/4th_tmp/forAOO/"+target_species+"_aoo"+str(i)+".csv", index = None)
        
            

### AOO 계산

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import math

AOO_pot_mix = []
Year_pot_mix = []
std_pot_mix = []
var_pot_mix = []
mean_pot_mix = []
AOO_nav = pd.DataFrame({"Year":[],"AOO":[]})

for samp in tqdm(range(0,30)):

    AOO(df_train = df_train_18,
    df_test = df_test ,
    targetnum=len(df_test),
    varlist=shap_var,Year_start=2007,Year_end=2021,
    target_species=target_species,bagstart=(samp)*50,bagnum=(samp+1)*50
    ,repstart=(samp)*50,repnum=(samp+1)*50,learningrate=0.7,to_csv=True)
    
    
    nav = {}
    ex = []
    for i in range(2007,2022):
        try:
            driver = webdriver.Chrome()
            driver.get('http://geocat.kew.org/')
            driver.find_element(By.XPATH,'/html/body/section[2]/div/header/a').click()
            time.sleep(0.5)
            driver.find_element(By.XPATH,'//*[@id="welcome"]/div[2]/ul/li[2]/a').click()
            time.sleep(0.5)
            driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div/div/div/ul/li[6]/a').click()
            time.sleep(0.5)


            # send_keys argument:absolute path of AOO csv
            driver.find_element(By.CSS_SELECTOR,"#add_source_container > div > div > div > ul > li.upload_source.selected > div > div.idle > div > form > input[type=file]").send_keys(smooth_path + target_species+"_aoo"+str(i)+".csv")


            time.sleep(12)
            # upload your file
            driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div/div/div/ul/li[6]/div/div[3]/a[2]').send_keys(Keys.ENTER)
            time.sleep(15)
            #check aoo text
            driver.find_element(By.XPATH,'//*[@id="toggle_analysis"]/span').click()
            time.sleep(0.5)
            # save aoo text
            nav[str(i)]=int(driver.find_element(By.CSS_SELECTOR,'#tools > div > div.center > div.center-tool > div.right > div.analysis > div > div.analysis_data > ul > li:nth-child(2) > p:nth-child(2)').text.replace(",","").split(".")[0])
            # quit
            driver.quit()
            print(i)
        except Exception as e:
            ex.append(i)
            driver.quit()
            print(i,"Error", e)
            pass
    

    for i in ex:
        try:
            driver = webdriver.Chrome()
            driver.get('http://geocat.kew.org/')
            driver.find_element(By.XPATH,'/html/body/section[2]/div/header/a').click()
            time.sleep(0.5)
            driver.find_element(By.XPATH,'//*[@id="welcome"]/div[2]/ul/li[2]/a').click()
            time.sleep(0.5)
            driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div/div/div/ul/li[6]/a').click()
            time.sleep(0.5)


            # send_keys 뒤에 절대경로 넣기
            driver.find_element(By.CSS_SELECTOR,"#add_source_container > div > div > div > ul > li.upload_source.selected > div > div.idle > div > form > input[type=file]").send_keys(smooth_path + target_species+"_aoo"+str(i)+".csv")


            time.sleep(15)
            #업로드시키기
            driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/div[3]/div[2]/div/div/div/div/ul/li[6]/div/div[3]/a[2]').send_keys(Keys.ENTER)
            time.sleep(20)
            #aoo 텍스트 따오기
            driver.find_element(By.XPATH,'//*[@id="toggle_analysis"]/span').click()
            time.sleep(0.5)
            #aoo 텍스트 저장
            nav[str(i)]=int(driver.find_element(By.CSS_SELECTOR,'#tools > div > div.center > div.center-tool > div.right > div.analysis > div > div.analysis_data > ul > li:nth-child(2) > p:nth-child(2)').text.replace(",","").split(".")[0])
            #창닫기
            driver.quit()
            print(i)
        except Exception as e:
            driver.quit()
            print(i,"Error Final", e)
            pass

    nav = dict(sorted(nav.items(), key=lambda x: x[0]))



    AOO_fin = pd.DataFrame({"Year":nav.keys(),"AOO":nav.values()})
    AOO_fin['Year'] = AOO_fin['Year'].astype(float)
    AOO_fin['AOO'] = AOO_fin['AOO'].astype(float)

    AOO_nav = pd.concat([AOO_nav,AOO_fin])


    ############# reduction rate
    Reduction_rate = pd.DataFrame({"Start_Year":[],"AOO":[]})
    Reduction_rate['Start_Year'] = list(range(2007,2013))

    try:
        Reduction_rate['AOO'][0] = (float(AOO_fin['AOO'][0]) - float(AOO_fin['AOO'][9]))/float(AOO_fin['AOO'][0])
        Reduction_rate['AOO'][1] = (float(AOO_fin['AOO'][1]) - float(AOO_fin['AOO'][10]))/float(AOO_fin['AOO'][1])
        Reduction_rate['AOO'][2] = (float(AOO_fin['AOO'][2]) - float(AOO_fin['AOO'][11]))/float(AOO_fin['AOO'][2])
        Reduction_rate['AOO'][3] = (float(AOO_fin['AOO'][3]) - float(AOO_fin['AOO'][12]))/float(AOO_fin['AOO'][3])
        Reduction_rate['AOO'][4] = (float(AOO_fin['AOO'][4]) - float(AOO_fin['AOO'][13]))/float(AOO_fin['AOO'][4])
        Reduction_rate['AOO'][5] = (float(AOO_fin['AOO'][5]) - float(AOO_fin['AOO'][14]))/float(AOO_fin['AOO'][5])

    except:
        pass

    AOO_pot_mix.append(Reduction_rate['AOO'].max())
    
    try:
        Year_pot_mix.append(int(Reduction_rate[Reduction_rate['AOO'] == Reduction_rate['AOO'].max()]['Start_Year'].values))
    except:
        Year_pot_mix.append(0)

    std_pot_mix.append(Reduction_rate['AOO'].std())
    var_pot_mix.append(Reduction_rate['AOO'].var())
    mean_pot_mix.append(Reduction_rate['AOO'].mean())


AOO_pot_mix = [0 if math.isnan(x) else x for x in AOO_pot_mix]
Year_pot_mix = [0 if math.isnan(x) else x for x in Year_pot_mix]
std_pot_mix = [0 if math.isnan(x) else x for x in std_pot_mix]
var_pot_mix = [0 if math.isnan(x) else x for x in var_pot_mix]
mean_pot_mix = [0 if math.isnan(x) else x for x in mean_pot_mix]




In [ ]:
df_end = pd.DataFrame({"AOO_pot_mix":AOO_pot_mix,
            "mean_pot_mix":mean_pot_mix,
            "std_pot_mix":std_pot_mix,
            'var_pot_mix':var_pot_mix,
            'Year_pot_mix':Year_pot_mix})

In [ ]:
df_end.to_csv("./Data/4th_tmp/"+key+"_interval.csv")

In [ ]:
AOO_nav

In [ ]:
AOO_nav.to_csv("./Data/4th_tmp/"+key+"_AOO_check.csv")

In [ ]:
df_end